# aid

aid is a collection of tools for interacting with UC Berkeley's AID (Aggressive IP Distribution) List.  The aid package consists of:

- aid module - python library for retrieving the AID list
- aid-iptables - cli script that generates IPTables Rules to block IPs contained in the AID List

## Using the aid module

First import the aid module

In [1]:
import aid
from pprint import pprint

### Get all AID entries for the past week

By default, get_aidlist() will return all AID List entries from the past week

In [2]:
aid_list = aid.entries()

get_aidlist returns a list of AidEntry namedtuples.

An AidEntry is a simple object (Named Tuple) with the following fields

In [3]:
print(aid.AIDEntry._fields)

('ip', 'tags', 'dst_port', 'last_seen_ts', 'first_seen_ts', 'service', 'seen_count')


In [4]:
print(aid_list[0])

entry = aid_list[0]
print()
print(entry.ip, entry.dst_port)

AidEntry(ip=IPv4Address('210.212.98.235'), tags=[], dst_port=3306, last_seen_ts='2015-10-13T06:52:41.000-07:00', first_seen_ts='2015-05-12T15:46:33.000-07:00', service='mysql', seen_count=39)

210.212.98.235 3306


### Filtering by service

The AID List can be filtered to only return a subset of services by passing a list to the services parameter.   The default value for services is "", and does apply any filtering by service.

In [5]:
aid_list_ssh = aid.entries(services=['ssh'])
aid_list_rdp = aid.entries(services=['rdp'])

aid_list_ssh_and_rdp = aid.entries(services=['ssh', 'rdp'])

print(aid_list_ssh[0])
print()
print("aid_list_ssh: {} entries\naid_list_rdp: {} entries\naid_list_ssh_and_rdp {} entries".format(
        len(aid_list_ssh), len(aid_list_rdp), len(aid_list_ssh_and_rdp)))

AidEntry(ip=IPv4Address('222.186.190.55'), tags=[], dst_port=22, last_seen_ts='2015-10-13T14:02:22.000-07:00', first_seen_ts='2015-09-07T19:35:27.000-07:00', service='ssh', seen_count=14)

aid_list_ssh: 94 entries
aid_list_rdp: 14 entries
aid_list_ssh_and_rdp 108 entries


### Filtering by date

The AID List can also be filtered to only include IP's detected since a start date.  The default value for start_date is 1 week.

In [6]:
aid_list_last_day = aid.entries(start_date='1 day')
aid_list_two_weeks = aid.entries(start_date='2 weeks')
aid_list_since_sept = aid.entries(start_date="2015-09-01")

print("aid_list_last_day: {}".format(len(aid_list_last_day)))
print("aid_list_two_weeks: {}".format(len(aid_list_two_weeks)))
print("aid_list_since_sept: {}".format(len(aid_list_since_sept)))

aid_list_last_day: 29
aid_list_two_weeks: 200
aid_list_since_sept: 453


### Filtering by Aggressiveness

Each time an IP generates an alert, it's "seen_count" is incremented.  This can be used to filter out less aggressive IPs.  The default value for seen_count is 10.

In [7]:
all_ips = aid.entries()
more_aggro_ips = aid.entries(seen_count=50)

print("all_ips: {}\naggro_ips: {}".format(len(all_ips), len(more_aggro_ips)))

all_ips: 141
aggro_ips: 68


### Combing Filters

All the above filtering techniques can be combined

In [8]:
combo_filter = aid.entries(services=['ssh', 'rdp'], start_date='1 month', seen_count=25)
print("combo_filter: {}".format(len(combo_filter)))

combo_filter: 144


### Getting Unique IP Addresses

If an IP generates alerts for multiple services (ssh and rdp for example) then that IP will have multiple entries in the aid list (one per service).  The aid modules also proves ips() which returns a list of unique IP addresses only.

In [9]:
all_entries = aid.entries()
unique_ips = aid.ips()

print("all_ips: {}\nunique_ips: {}".format(len(all_ips), len(all_entries)))
print("\n")
for ip in unique_ips[:5]:
    print(ip)

all_ips: 141
unique_ips: 141


14.141.150.186
23.95.82.74
23.234.29.139
27.221.10.43
36.72.228.72
